In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer2 import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
locs += np.array([0,0,-0.70])
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7004
INFO:drake:Meshcat listening for connections at http://localhost:7005


In [5]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat1.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

In [7]:
visualizer.meshcat2.Delete("/prm")

In [8]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plotting_fn(nodes, adjacency_list, width):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = Rgba(0.0, 0.0, 1, 1))
            plt_idx +=1
            
plotting_fn_handle = partial(plotting_fn, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)

PRM = prm.PRM( 
            limits_t,
            num_points = 500,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

# PRM.add_start_end(start, target)
PRM.plot()
# path, sp_length = PRM.find_shortest_path()

# mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
# mat.wireframeLinewidth = 2.0
# num_waypoints = len(path)
# for idx in range(num_waypoints-1):
#     vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
# traj= utils.PWLinTraj(path, 5.0)

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Samples 120
[PRM] Samples 150
[PRM] Samples 180
[PRM] Samples 210
[PRM] Samples 240
[PRM] Samples 270
[PRM] Samples 300
[PRM] Samples 330
[PRM] Samples 360
[PRM] Samples 390
[PRM] Samples 420
[PRM] Samples 450
[PRM] Samples 480
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80
[PRM] Nodes connected: 100
[PRM] Nodes connected: 120
[PRM] Nodes connected: 140
[PRM] Nodes connected: 160
[PRM] Nodes connected: 180
[PRM] Nodes connected: 200
[PRM] Nodes connected: 220
[PRM] Nodes connected: 240
[PRM] Nodes connected: 260
[PRM] Nodes connected: 280
[PRM] Nodes connected: 300
[PRM] Nodes connected: 320
[PRM] Nodes connected: 340
[PRM] Nodes connected: 360
[PRM] Nodes connected: 380
[PRM] Nodes connected: 400
[PRM] Nodes connected: 420
[PRM] Nodes connected: 440
[PRM] Nodes connected: 460
[PRM] Nodes connected: 480


## Set up the sliders so we can move the plant around manually

In [9]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

In [10]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


In [11]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, np.zeros(2),
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

Time to construct line certifier = 18.118589401245117s


In [12]:
PRM.nodes

array([[-4.76514150e-01, -4.70606700e-01],
       [ 1.56579325e-01, -8.43075676e-01],
       [-2.26373902e-01, -1.01800866e+00],
       [ 3.29087404e-01, -6.53630585e-01],
       [-4.34629391e-01, -7.67058348e-01],
       [-5.56087857e-01, -4.29481990e-01],
       [-5.81575258e-01, -9.52459665e-01],
       [-4.75566811e-01, -5.13882698e-01],
       [-4.27403369e-01, -7.03534358e-01],
       [ 1.49183420e-01, -8.90138726e-01],
       [-5.92162189e-01,  1.39715487e-01],
       [ 3.62622679e-01, -8.21490639e-01],
       [-3.33094270e-01, -3.95365247e-01],
       [-3.71730978e-01, -2.39759187e-01],
       [ 3.19420584e-01,  1.23407937e-01],
       [-2.99098241e-01, -5.09579223e-01],
       [ 3.07550289e-01,  2.59396117e-02],
       [-1.62055505e-01, -5.04244498e-01],
       [-4.82530445e-01, -6.01619761e-01],
       [-7.67184638e-02, -4.36813188e-01],
       [-1.44731196e-01, -8.18605677e-01],
       [-3.89962949e-01, -5.93032731e-01],
       [-3.85880862e-01, -8.68434737e-01],
       [ 2.

In [14]:
s0 = PRM.nodes[PRM.adjacency_list[0][0]]
uncertified_edges = []
print(s0)
t0_tot = time.time()
for item_num, item in enumerate(PRM.adjacency_list):
    s0 = PRM.nodes[item[0]]
    if (item_num % 100 == 0):
        print(f"Doing {item_num}/{len(PRM.adjacency_list)}")
    for idx1 in item[1:]:
        s1 = PRM.nodes[idx1]
        t0 = time.time()
        safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1)
        t1 = time.time()
#         print(f"Edge ({item[0]}, {    for idx1 in item[1:]:
# }) is {safe}")
#         print(f"Certified in {t1-t0}s")
#         print()
        if not safe:
            uncertified_edges.append([item[0], idx1])
t1_tot = time.time()
print(f"Certification in {t1_tot-t0_tot}s")

[ 0.26672149 -0.13035819]
Doing 0/500
Doing 100/500
Doing 200/500
Doing 300/500
Doing 400/500
Certification in 1920.2652184963226s


In [15]:
print(len(uncertified_edges))

91


In [ ]:

line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kConstant, np.zeros(2),
                                set(), VerificationOption()
                               )
s0 = PRM.nodes[PRM.adjacency_list[0][0]]
uncertified_edges = []
print(s0)
t0_tot = time.time()
for item_num, item in enumerate(PRM.adjacency_list):
    s0 = PRM.nodes[item[0]]
    if (item_num % 100 == 0):
        print(f"Doing {item_num}/{len(PRM.adjacency_list)}")
    for idx1 in item[1:]:
        s1 = PRM.nodes[idx1]
        t0 = time.time()
        safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1)
        t1 = time.time()
#         print(f"Edge ({item[0]}, {idx1}) is {safe}")
#         print(f"Certified in {t1-t0}s")
#         print()
        if not safe:
            uncertified_edges.append([item[0], idx1])
t1_tot = time.time()
print(f"Certification in {t1_tot-t0_tot}s")

[-0.47651415 -0.4706067 ]
Doing 0/500
Doing 100/500
Doing 200/500
Doing 300/500
